In [1]:
# using Pkg
# Pkg.add("JSON") 

In [2]:
using DataStructures

In [3]:
lens = Dict{Tuple{Int64, Int64}, Real}()

# for i in 1:4
#     for j in 1:4
#         ind = (i, j)
#         if i == j
#             lens[ind] = 0
#         else
#             lens[ind] = 1
#         end
#     end
# end

eps = 1e-6

1.0e-6

In [4]:
function cross_prod(v::AbstractVector{<:Real}, w::AbstractVector{<:Real})
    return v[1] * w[2] - v[2] * w[1]
end

function quadratic_equation_solve(a::Real, b::Real, c::Real)
    D = b * b - 4 * a * c
    return [(-b - sqrt(D)) / (2 * a), (-b + sqrt(D)) / (2 * a)]
end

function dist(v::AbstractVector{<:Real}, w::AbstractVector{<:Real})
    return sqrt((v[1] - w[1]) * (v[1] - w[1]) + (v[2] - w[2]) * (v[2] - w[2]))
end

function dist3(v::AbstractVector{<:Real}, w::AbstractVector{<:Real})
    return sqrt((v[1] - w[1]) * (v[1] - w[1]) + (v[2] - w[2]) * (v[2] - w[2]) + (v[3] - w[3]) * (v[3] - w[3]))
end

dist3 (generic function with 1 method)

In [5]:
function intersection(w1::Tuple{AbstractVector{<:Real}, <:Real}, w2::Tuple{AbstractVector{<:Real}, <:Real})
    x1 = w1[1][1]
    x2 = w2[1][1]
    y1 = w1[1][2]
    y2 = w2[1][2]
    r1 = w1[2]
    r2 = w2[2]

    a = 2 * (x2 - x1)
    b = 2 * (y2 - y1)
    c = (x1 * x1 - x2 * x2 + y1 * y1 - y2 * y2 - r1 * r1 + r2 * r2)
    if abs(b) < eps
        x = -c / a
        ys = quadratic_equation_solve(1, -2 * y1, y1 * y1 - r1 * r1 + (x - x1) * (x - x1))
        xs = [x, x]
    else
        xs = quadratic_equation_solve(((a * a) / (b * b)) + 1, -2 * x1 + 2 * (a / b) * (y1 + c / b), x1 * x1 - r1 * r1 + (y1 + c / b) * (y1 + c / b))
        ys = [(-c - a * xs[1]) / b, (-c - a * xs[2]) / b]
    end
    # println((xs[1] - x1) * (xs[1] - x1) + (ys[1] - y1) * (ys[1] - y1) - r1 * r1, ' ', (xs[1] - x2) * (xs[1] - x2) + (ys[1] - y2) * (ys[1] - y2) - r2 * r2)
    return [[xs[1], ys[1]], [xs[2], ys[2]]]
end

intersection (generic function with 1 method)

In [6]:
function make_next(A::Tuple{AbstractVector{<:Real}, Int64}, B::Tuple{AbstractVector{<:Real}, Int64}, C::Tuple{AbstractVector{<:Real}, Int64}, edge::Tuple{Int64, Int64})
    left = edge[1]
    right = edge[2]
    vertexes = [A, B, C]
    ver_nums = [A[2], B[2], C[2]]
    V1 = vertexes[(findfirst(x -> x == left, ver_nums))]
    V2 = vertexes[(findfirst(x -> x == right, ver_nums))]
    V3 = vertexes[(findfirst(x -> !(x == left || x == right), ver_nums))]
    num_D = 10 - A[2] - B[2] - C[2]
    V1D = lens[(V1[2], num_D)]
    V2D = lens[(V2[2], num_D)]
    Ds = intersection((V1[1], V1D), (V2[1], V2D))
    D1 = Ds[1]
    D2 = Ds[2]
    V1V2 = [V2[1][1] - V1[1][1], V2[1][2] - V1[1][2]]
    V1D1 = [D1[1] - V1[1][1], D1[2] - V1[1][2]]
    V1V3 = [V3[1][1] - V1[1][1], V3[1][2] - V1[1][2]]
    D = (D1, num_D)
    if cross_prod(V1V2, V1D1) * cross_prod(V1V2, V1V3) > 0
        D = (D2, num_D)
    end
    return [V1, V2, D]
end

make_next (generic function with 1 method)

In [7]:
# A = ([0, 0], 1)
# B = ([1, 0], 2)
# C = ([1 / 2, sqrt(3) / 2], 3)
# edge = (1, 3)
# println(make_next(A, B, C, edge))

In [8]:
function add_point(P::Tuple{AbstractVector{<:Real}, Int64}, points::AbstractVector{Tuple{AbstractVector{Real}, Int64}}, t::Real)
    if (dist(P[1], [0, 0])) > t
        return false
    end
    for check in points
        if (dist(P[1], check[1]) < eps && P[2] == check[2])
            return false
        end
    end
    push!(points, P)
    return true
end

add_point (generic function with 1 method)

In [19]:
function get_all_points(A::AbstractVector{<:Real}, B::AbstractVector{<:Real}, C::AbstractVector{<:Real}, D::AbstractVector{<:Real}, t::Real)
    vertexes = [A, B, C, D]
    for i in 1:4
        for j in 1:4
            ind = (i, j)
            if i == j
                lens[ind] = 0
            else
                lens[ind] = dist3(vertexes[i], vertexes[j])
            end
        end
    end

    Ap = (A[1:2], 1)
    Bp = (B[1:2], 2)
    Cp = (C[1:2], 3)
    start = [Ap, Bp, Cp]
    points = Base.Vector{Tuple{AbstractVector{Real}, Int64}}()
    push!(points, Ap)
    push!(points, Bp)
    push!(points, Cp)
    q = Queue{Tuple{AbstractVector{Tuple{AbstractVector{Real}, Int64}}, Tuple{Int64, Int64}}}()
    enqueue!(q, (start, (0, 0)))
    start_time = time_ns()
    while length(q) != 0 && (time_ns() - start_time) < 3e10
        info = dequeue!(q)
        face = info[1]
        last_edge = info[2]
        edges = [(face[1][2], face[2][2]), (face[1][2], face[3][2]), (face[2][2], face[3][2])]
        edges = filter(x -> x != last_edge, edges)
        faces = Base.Vector{Tuple{AbstractVector{Tuple{AbstractVector{<:Real}, Int64}}, Tuple{Int64, Int64}}}()
        for edge in edges
            push!(faces, (make_next(face[1], face[2], face[3], edge), edge))
        end
        for info_it in faces
            face_it = info_it[1]
            edge_if = info_it[2]
            flag = false
            for point_tup in face_it
                flag = flag || add_point(point_tup, points, t)
            end
            if flag
                enqueue!(q, info_it)
            end
        end
    end
    return points
end

get_all_points (generic function with 1 method)

In [20]:
function get_geodesics(points::AbstractVector{Tuple{AbstractVector{Real}, Int64}}, precision::Int64)
    result = Dict{Real, Int64}()
    for A in points
        for B in points
            if A[2] == B[2]
                continue
            end
            geodesic_len = round(dist(A[1], B[1]), digits=precision)
            result[geodesic_len] = get(result, geodesic_len, 0) + 1
        end
    end
    return result
end


get_geodesics (generic function with 1 method)

In [28]:
A = [0, 0, 0]
B = [1, 0, 0]
C = [1 / 2, sqrt(3) / 2, 0]
D = [1 / 2, sqrt(3) / 6, sqrt(6) / 2]
count_lens = Dict{Real, Int64}()

points = get_all_points(A, B, C, D, 10)

# for double_t in 1:6
#     t = double_t / 2
#     points = get_all_points(A, B, C, D, t)
#     count_lens[t] = length(get_geodesics(points, 3))
#     println("done ", t)
#     println(count_lens[t])
# end
# println(1)

10895-element Vector{Tuple{AbstractVector{Real}, Int64}}:
 ([0, 0], 1)
 ([1, 0], 2)
 ([0.5, 0.8660254037844386], 3)
 ([0.5, -1.2583057392117916], 4)
 ([-0.839724735885168, 1.062165571498115], 4)
 ([1.8397247358851678, 1.0621655714981149], 4)
 ([-0.727272727272727, -0.6863485850246136], 3)
 ([1.7272727272727273, -0.6863485850246136], 3)
 ([-0.9580316741191828, -0.2866623647854667], 2)
 ([0.2692410531535447, 1.8390363535945187], 2)
 ⋮
 ([-5.671408054114878, 1.3107781965764438], 4)
 ([-1.8939073309816419, -0.09377825944709287], 3)
 ([-2.1806549944156575, 2.3439602757418037], 3)
 ([-4.572332105983994, 1.059246534558979], 1)
 ([-2.604562438443158, 2.5264465631631126], 1)
 ([-1.3237572494091212, -1.2335160131091278], 2)
 ([0.23023952681044194, -2.4587332867947254], 2)
 ([-1.12259599652307, -5.027523313275866], 4)
 ([0.713859890861336, -3.076400752937461], 4)

In [12]:
print(count_lens)

Dict{Real, Int64}()

In [29]:
using JSON

io = open("points.json", "w");
JSON.print(io, points, 2)
close(io);

In [14]:
# using JSON

# io = open("reg_points.json", "w");
# JSON.print(io, count_lens, 2)
# close(io);

In [15]:
# using JSON

# io = open("res.json", "w");
# res = Dict{Tuple{Real, Real}, Int64}()
# for point in points
#     res[(point[1][1], point[1][2])] =point[2]
# end
# JSON.print(io, res, 2)
# close(io);

In [16]:
# A1 = [0, 0, 0]
# B1 = [1, 0, 0]
# C1 = [1 / 2, sqrt(3) / 2, 0]
# D1 = [1 / 2, sqrt(3) / 6, sqrt(6) / 2]
# t = 5

# points2 = get_all_points(A1, B1, C1, D1, t)

In [17]:
# using JSON

# io = open("res2.json", "w");
# res = Dict{Tuple{Real, Real}, Int64}()
# for point in points2
#     res[(point[1][1], point[1][2])] = point[2]
# end
# JSON.print(io, res, 2)
# close(io);